## Demo.ipynb

Written by Taiob Ali  
SqlWorldWide.com

This script will create:

- One resource group
- Three logical SQL server
- Two sample databases

Credit:

- [Azure PowerShell samples for Azure SQL Database and Azure SQL Managed Instance](https://docs.microsoft.com/en-us/azure/azure-sql/database/powershell-script-content-guide?tabs=single-database)
- [PowerShell One Liner: Get External/Public IP Address](https://gallery.technet.microsoft.com/scriptcenter/Get-ExternalPublic-IP-c1b601bb)
- [Azure SQL Database and Azure Synapse IP firewall rules](https://docs.microsoft.com/en-us/azure/azure-sql/database/firewall-configure#manage-firewall-rules-using-azure-cli)

PowerShell 7.x and later is the recommended version of PowerShell for use with Azure PowerShell on all platforms.<br>Check your PowerShell version and update as required

In [ ]:
$PSVersionTable.PSVersion

Install the Azure PowerShell module if you have not already

In [ ]:
Install-Module -Name Az

If you've disabled module autoloading, manually import the module with `Import-Module -Name Az`. Because of the way the module is structured, this can take a few seconds.

In [ ]:
Import-Module -Name Az
Import-Module -Name Az.Accounts

Sign in to Azure

In [ ]:
Connect-AzAccount

If you need to see the list of your subscription

<span style="color: #6a9955;">$SubscriptionList=Get-AzSubscription</span><span style="color: rgb(106, 153, 85);">$SubscriptionList</span>

Use below code if you have multiple subscription and you want to use a particular one

In [ ]:
Set-AzContext -SubscriptionId '6f8db000-8416-43f0-a2db-cbfb7c945982'
#Set-AzContext -SubscriptionId '18d92f52-ac34-4379-ab8b-5a5106f1c54e'

Declare Variables

In [ ]:
$resourceGroupName = "rgeu2hadrdemo"
$primaryLocation = "East US" 
$secondaryLocation = "West US"
$primaryServerName = "primaryserver1004"
$secondaryServerName = "secondaryserver1004"
$readScaleoutServerName = "readscaleoutserver1004"
$databaseName = "sqlDatabase1004"
$databaseNameReadScale = "sqlDatabaseReadScale1004"
$failoverGroupName = "failovergroup1004"
$ipinfo = Invoke-RestMethod http://ipinfo.io/json 
$startip = $ipinfo.ip
$endip = $ipinfo.ip 
$adminlogin = "taiob"
$password = Get-Content "C:\password.txt"

Check if Resource group exists. If exist delete the resource group.

Create a new Resource group

In [ ]:
$resGrpChk = Get-AzResourceGroup `
    -Name $resourceGroupName `
    -ev notPresent `
    -ea 0

if ($resGrpChk) {  
    #Delete resource group
    Remove-AzResourceGroup `
        -Name $resourceGroupName -Confirm   
    Write-Host 'Resource group deleted' `
        -fore white `
        -back green
}

New-AzResourceGroup `
    -Name $resourceGroupName `
    -Location $primaryLocation    
Write-Host 'Resource group created' `
    -fore white `
    -back green

Create a Azure SQL Server at Primary Region (East US 2)

In [ ]:
New-AzSqlServer `
  -ResourceGroupName $resourceGroupName `
  -ServerName $primaryServerName `
  -Location $primaryLocation `
  -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
  -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force)) 

Configure Firewall rule for the above server

In [ ]:
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $primaryServerName `
  -FirewallRuleName "TaiobDemoMachine" `
  -StartIpAddress $startip `
  -EndIpAddress $endip

# This is done to allow access to Azure Services
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $primaryServerName `
  -FirewallRuleName "AllowAzureService" `
  -StartIpAddress 0.0.0.0 `
  -EndIpAddress 0.0.0.0

Create a Azure SQL Server at Secondary Region (West US)

<span style="font-size: 14px;"><a href="https://docs.microsoft.com/en-us/azure/best-practices-availability-paired-regions" data-href="https://docs.microsoft.com/en-us/azure/best-practices-availability-paired-regions" title="https://docs.microsoft.com/en-us/azure/best-practices-availability-paired-regions">https://docs.microsoft.com/en-us/azure/best-practices-availability-paired-regions</a></span>

In [ ]:
New-AzSqlServer -ResourceGroupName $resourceGroupName `
  -ServerName $secondaryServerName `
  -Location $secondaryLocation `
  -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
  -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force))

Configure Firewall rule for the above server

In [ ]:
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $secondaryServerName  `
  -FirewallRuleName "TaiobDemoMachine" `
  -StartIpAddress $startip `
  -EndIpAddress $endip

# This is done to allow access to Azure Services
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $secondaryServerName `
  -FirewallRuleName "AllowAzureService" `
  -StartIpAddress 0.0.0.0 `
  -EndIpAddress 0.0.0.0

Create a Azure SQL Serer to demo Read Scale-Out

In [ ]:
New-AzSqlServer `
  -ResourceGroupName $resourceGroupName `
  -ServerName $readScaleoutServerName `
  -Location $primaryLocation `
  -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
  -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force))

Configure Firewall rule for the above server

In [ ]:
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $readScaleoutServerName `
  -FirewallRuleName "TaiobDemoMachine" `
  -StartIpAddress $startip `
  -EndIpAddress $endip

# This is done to allow access to Azure Services
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $readScaleoutServerName `
  -FirewallRuleName "AllowAzureService" `
  -StartIpAddress 0.0.0.0 `
  -EndIpAddress 0.0.0.0

Creating an Azure SQL Database with sample database AdventureWorksLT

Puchase Model: vCpu<br>Service Tier : General Purpose

In [ ]:
New-Azsqldatabase  `
  -ResourceGroupName $resourceGroupName `
  -ServerName $primaryServerName `
  -DatabaseName $databaseName `
  -SampleName "AdventureWorksLT" `
  -RequestedServiceObjectiveName "GP_Gen5_2" `
  -MaxSizeBytes 5368709120

Creating an Azure SQL Database with sample database AdventureWorksLT

Puchase Model: vCpu<br>Service Tier : Business Critical

In [ ]:
New-Azsqldatabase  `
  -ResourceGroupName $resourceGroupName `
  -ServerName $readScaleoutServerName `
  -DatabaseName $databaseNameReadScale `
  -SampleName "AdventureWorksLT" `
  -RequestedServiceObjectiveName "BC_Gen5_2" `
  -MaxSizeBytes 5368709120

# Demo Starts here

## Demo : Automated backup

Open the Azure portal and show lon-term retention (LTR) backups is not enable by default

```
SQL Server-->Data management-->Backups
```

You will only see Pont-in-time restore (PITR)

Get all long term retention (LTR) policies within a server

<span style="font-size: 14px;">https://docs.microsoft.com/en-us/powershell/module/az.sql/get-azsqldatabasebackuplongtermretentionpolicy?view=azps-5.6.0&amp;viewFallbackFrom=azps-4.3.0</span>

In [ ]:
$ltrPolicies = Get-AzSqldatabase `
  -ResourceGroupName $resourceGroupName `
  -ServerName $primaryServerName `
  -DatabaseName $databaseName `
| Get-AzSqldatabaseLongTermRetentionPolicy

Write-Output $ltrPolicies

Set up long-term retention backup 

<span style="font-size: 14px;">https://docs.microsoft.com/en-us/powershell/module/az.sql/set-azsqldatabasebackuplongtermretentionpolicy?view=azps-5.6.0</span>  

Create LTR policy with below specification

WeeklyRetention = 12 weeks  
MonthlyRetention = 0  
YearlyRetetion = 5 years  
WeekOfYear = 16

In [ ]:
Set-AzSqldatabaseBackupLongTermRetentionPolicy `
  -ServerName $primaryServerName `
  -DatabaseName $databaseName `
  -ResourceGroupName $resourceGroupName `
  -WeeklyRetention P12W `
  -YearlyRetention P5Y `
  -WeekOfYear 16

Check the long-term retention (LTR) policies within a server

In [ ]:
$ltrPolicies = Get-AzSqldatabase `
  -ResourceGroupName $resourceGroupName `
  -ServerName $primaryServerName `
  -DatabaseName $databaseName `
| Get-AzSqldatabaseLongTermRetentionPolicy

Write-Output $ltrPolicies

Do not need to run the code during demo

Remove LTR policy from a database<br>

In [ ]:
Set-AzSqlDatabaseBackupLongTermRetentionPolicy `
  -ServerName $primaryServerName `
  -DatabaseName $databaseName `
  -ResourceGroupName $resourceGroupName `
  -RemovePolicy

## Demo : Built in HA/DR (with NO Configuration)

This demo code is copied from:  
[Basic high availability in Azure SQL Database](https://github.com/microsoft/sqlworkshops-azuresqlworkshop/blob/master/azuresqlworkshop/05-Availability/basic-ha/basic-ha.ipynb)

Start a workload using [ostress](https://www.microsoft.com/en-us/download/details.aspx?id=4511)  

Once the failover is done press ctrl+c to stop the ostress

```
ostress.exe -S"primaryServer1004.database.windows.net" -Q"SELECT COUNT(*) FROM SalesLT.Customer" -U"taiob" -d"sqlDataBase1004" -n1 -r50000
```

Initiate a failover  
Since 'sqlDatabase1004' is in 'General Purpose' service tier it will take longer time compare to 'Business Critical'

In [ ]:
$Start = Get-Date
Invoke-AzSqlDatabaseFailover `
  -ResourceGroupName $resourceGroupName `
  -ServerName $primaryServerName `
  -DatabaseName $databaseName
$end = Get-Date
$Runtime = $end - $Start
$Runtime

Start a workload using [ostress](https://www.microsoft.com/en-us/download/details.aspx?id=4511)

Once the failover is done press ctrl+c to stop the ostress

```
ostress.exe -S"readscaleoutserver1004.database.windows.net" -Q"SELECT COUNT(*) FROM SalesLT.Customer" -U"taiob" -d"sqlDataBaseReadScale1004" -n1 -r50000 -o"C:\Users\taiob\AppData\Local\Temp\output\ostress2.log"
```

Initiate a failover  
Since 'sqlDatabaseReadScale1004' is in 'Business Critical' service tier it will very quick

In [ ]:
$Start = Get-Date
Invoke-AzSqlDatabaseFailover `
  -ResourceGroupName $resourceGroupName `
  -ServerName $readScaleoutServerName `
  -DatabaseName $databaseNameReadScale
$end = Get-Date
$Runtime = $end - $Start
$Runtime

## Demo : Active Geo-Replication

Setting up Active Geo-Replication for sqlDatabase1004  
Between 'primaryserver1004.database.windows.net' and 'secondaryserver1004.database.windows.net'

While this is running go to Azure portal and see that it is being setup

In [ ]:
$database = 
Get-AzSqldatabase `
  -DatabaseName $databaseName `
  -ResourceGroupName $resourceGroupName `
  -ServerName $primaryServerName

$database | `
New-AzSqldatabaseSecondary `
  -PartnerResourceGroupName $resourceGroupName `
  -PartnerServerName $secondaryServerName `
  -AllowConnections "All"

Perform a planned failover (Switches a secondary database to be primary in order to initiate failover)</span>

<span style="font-size: 14px;">https://docs.microsoft.com/en-us/powershell/module/az.sql/set-azsqldatabasesecondary?view=azps-5.6.0</span>

In [ ]:
$database = 
Get-AzSqlDatabase `
  -DatabaseName $databaseName `
  -ResourceGroupName $resourceGroupName `
  -ServerName $secondaryServerName
$database | Set-AzSqlDatabaseSecondary -PartnerResourceGroupName $resourceGroupName -Failover

Go to Portal and see the new primary/seconadary for database sqlDatabase1004

Failback for next demo

In [ ]:
Set-AzsqldatabaseSecondary `
  -DatabaseName $databaseName `
  -PartnerResourceGroupName $resourceGroupName `
  -ServerName $primaryServerName `
  -ResourceGroupName $resourceGroupName `
  -Failover

## Demo : Auto-Failover Group

Create a Failover group

At this point Failover group is empty and does not contain any databases

In [ ]:
$failovergroup = 
New-AzSqldatabaseFailoverGroup `
    -ResourceGroupName $resourceGroupName `
    -ServerName $primaryServerName `
    -PartnerServerName $secondaryServerName `
    -FailoverGroupName $failoverGroupName `
    -GracePeriodWithDataLossHours 2

$failovergroup

Add database to the Failover gorup created above

In [ ]:
$failoverGroup = 
Get-AzSqldatabase `
  -ResourceGroupName $resourceGroupName `
  -ServerName $primaryServerName `
  -DatabaseName $databaseName | `
Add-AzSqldatabaseToFailoverGroup `
  -ResourceGroupName $resourceGroupName `
  -ServerName $primaryServerName `
  -FailoverGroupName $failoverGroupName 

$failovergroup

Go to the portal and show the failover group '<span style="font-size: 12px; white-space: pre-wrap;">failovergroup1004'</span>

<span style="font-size: 12px; white-space: pre-wrap;">ServerName--&gt; Data Mangement --&gt;Failover groups</span>

<span style="font-size: 12px; white-space: pre-wrap;">Two tabs</span>

- <span style="font-size: 12px; white-space: pre-wrap;">Configuration details</span>
- <span style="font-size: 12px; white-space: pre-wrap;">Databases within group</span>

Copy the endpoints, connect to the server, show server names

Initiate a planned failover

In [ ]:
Switch-AzSqldatabaseFailoverGroup `
  -ResourceGroupName $resourceGroupName  `
  -ServerName $secondaryServerName `
  -FailoverGroupName $failoverGroupName

Gets a specific Azure SQL Database Failover Group or lists the Failover Groups on a server.  
Either server in the Failover Group may be used to execute the command.  
The returned values will reflect the state of the specified server with respect to the Failover Group.

In [ ]:
Get-AzSqldatabaseFailoverGroup `
  -ResourceGroupName $resourceGroupName  `
  -ServerName $primaryServerName 
Get-AzSqldatabaseFailoverGroup `
  -ResourceGroupName $resourceGroupName  `
  -ServerName $secondaryServerName 

## Demo : Read Scale-Out

Premium and Business Critical service tier is automatically Enabled<br>Basic, Standard, and General Purpose service tiers does not include this feature

Checking current setting for Read Scale-Out

In [ ]:
 $isReadEnabled = 
Get-AzSqldatabase `
  -ResourceGroupName $resourceGroupName `
  -DatabaseName $databaseNameReadScale `
  -ServerName $readScaleoutServerName 

Write-host($isReadEnabled.ReadScale)

Use ReadSacleOut.sql show the built in Read Sclae-Out feature

Clean up by removing resource group name

In [ ]:
Remove-AzResourceGroup -ResourceGroupName $resourceGroupName -Force